In [26]:
import pandas as pd
import shapely
import numpy as np
import sys
import torch
import argparse
import random
import math
import os
import warnings
import pickle
from gensim.models import word2vec
from shapely.geometry import LineString, Polygon
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler# 好处在于可以保存训练集中的参数（均值、方差）
from scipy.stats import stats
#from torch.utils.data import DataLoader
#from torch.optim import lr_scheduler
from tqdm import tqdm
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
pd.options.display.max_columns = 999
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x))
import gc
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Prepare test data
- input should be list of "Bauteil" and their rotation
- all the features of "Bauteil" should not be new here, including the feature after normalization.
- we write the function for single test record

## steps 
- load all norm feature
- create part embedding and add them to features
- transform test data so that it can be used in the neural model

## part embedding

In [19]:
# load all the features after norm
data = pd.read_pickle('../results/all_norm_feature.pkl')

In [63]:
# add part embedding
# parts embedding, not tested jet.

# feature selection
df2 = data[['norm_area', 'length', 'area quote', 'convex area quote', 'convex area', 'area verhaltnis', 'centroid x/width',
        'centroid y/height', 'num_of_corr', 'A/L']]
df2 = df2.drop_duplicates()
df2.reset_index(inplace= True, drop=True)
df2['type'] = df2.index
# set container and meanshift cluster model
from sklearn.cluster import MeanShift
df_cluster = pd.DataFrame(df2['type'], index=df2.index)
ms = MeanShift()
# level x
for i in range(len(df2.columns)):
    _, items = get_combinations(i+1, df2.columns, si = 0, com = [])
    for col in tqdm(items):
        if col == 'type':
            continue
        clustering = ms.fit(df2[col])
        df_cluster.loc[:, 'clustering after ' + str(col)] = clustering.labels_
# only take the cluster which menge between 10 and 30
out = []
df_cluster.apply(lambda x: [out.append(df2['type'][x == i]) for i, j in zip(x.value_counts().index, x.value_counts().tolist()) if j < 30 and j > 10])
# change data type
out = [i.tolist() for i in out]
# create sentences from the cluster
sentences = []
for i in tqdm(out):
    num_sents = random.randint(5, 20)
    len_sents = random.randint(min(len(i), 5), min(len(i), 20))
    counter = 0
    for j in range(num_sents):
        tmp = []
        counter = 0
        while counter < len_sents:
            chosen = random.choice(i)
            if str(chosen) in tmp:
                continue
            tmp.append(str(chosen))
            counter = counter + 1
        sentences.append(tmp)
# train w2v model
model = word2vec.Word2Vec(sentences, min_count = 5, size = 8)
# merge back to dataframe on length
df2['wv'] = df2['type'].map(lambda x: model.wv[str(x)])
data = data.merge(df2[['length', 'wv']], how = 'left', on = 'length')

100%|██████████| 1/1 [00:00<00:00, 10.66it/s]


type                                                                                                                                                                                                          []
clustering after ['norm_area']                                                                                                                                                                [None, None, None]
clustering after ['length']                                                                                                                                                                                   []
clustering after ['area quote']                                                                                                                                                                           [None]
clustering after ['convex area quote']                                                                                                                              

100%|██████████| 3177/3177 [00:01<00:00, 1917.20it/s]


In [73]:
df2['wv0'] = df2['wv'].map(lambda x: x[0])
df2['wv1'] = df2['wv'].map(lambda x: x[1])
df2['wv2'] = df2['wv'].map(lambda x: x[2])
df2['wv3'] = df2['wv'].map(lambda x: x[3])
df2['wv4'] = df2['wv'].map(lambda x: x[4])
df2['wv5'] = df2['wv'].map(lambda x: x[5])
df2['wv6'] = df2['wv'].map(lambda x: x[6])
df2['wv7'] = df2['wv'].map(lambda x: x[7])

In [86]:
data[]

,Jobid,Rot,Left down,right down,score,bounding box,Shapely Polygon,Area,length,area quote,convex area quote,convex area,area verhaltnis,centroid x/width,centroid y/height,width/height,num_of_corr,r1,r2,A/L,Left_down_x,Left_down_y,Right_down_y,norm_area,norm_left_down_x,norm_left_down_y,norm_right_down_y,wv_x,wv0,wv1,wv2,wv3,wv4,wv5,wv6,wv7,wv_y,wv
0,792-0_10_0,90.000,"(93.0, 1.0)","(93.0, 377.0)",0.000,"(0.0, 0.0)","POLYGON ((159 49, 167 44, 168 44, 308 44, 308 ...",20525.000,0.847,-0.514,0.512,1.266,-0.965,0.733,2.238,0.560,0.910,0.000,1.000,0.315,93.000,1.000,377.000,0.916,-1.286,-1.994,-1.400,"[-0.24003835, -0.3264453, -0.51530653, 1.51531...",-0.240,-0.326,-0.515,1.515,0.027,0.566,-0.648,0.239,"[0.33473504, -1.7669557, -0.3693448, 0.8048883...","[0.74889624, -2.125927, -1.3886113, -0.3797980..."
1,792-0_10_1,0.000,"(213.0, 96.0)","(213.0, 254.0)",4.000,"(0.0, 0.0)","POLYGON ((199 158, 93 158, 66 132, 0 132, 0 13...",18230.000,0.580,-0.789,0.560,1.192,-1.334,-1.092,0.347,-0.497,-0.493,1.000,0.000,0.239,213.000,96.000,254.000,0.608,-0.992,-1.772,-1.671,"[-0.10381872, -0.3241408, -0.5326116, 1.695041...",-0.104,-0.324,-0.533,1.695,-0.222,0.483,-0.767,0.280,"[0.49004108, -1.522208, -0.18397297, 0.6107793...","[1.3287598, -2.0762594, -1.2859267, -0.8665443..."
2,792-0_10_2,90.000,"(139.0, 311.0)","(139.0, 494.0)",4.000,"(0.0, 0.0)","POLYGON ((20 138, 19 138, 17 138, 16 138, 0 13...",12481.500,0.339,-1.070,-1.456,-0.100,-0.364,-1.348,-0.681,-0.503,0.008,0.000,1.000,-0.461,139.000,311.000,494.000,-0.163,-1.173,-1.271,-1.142,"[-0.49517643, 0.5537861, -1.263407, 2.170962, ...",-0.495,0.554,-1.263,2.171,-0.567,0.851,-0.748,-0.492,"[0.43932813, -1.4193826, -1.1127329, 1.560613,...","[0.64179367, -2.2796211, -1.4502786, 0.556967,..."
3,792-0_10_3,0.000,"(234.0, 453.0)","(234.0, 581.0)",4.000,"(0.0, 0.0)","POLYGON ((150 128, 149 128, 149 92, 149 91, 12...",14630.000,-0.032,-1.078,-1.227,0.252,-0.577,0.396,-1.971,-0.316,0.409,1.000,0.000,0.322,234.000,453.000,581.000,0.125,-0.940,-0.939,-0.950,"[-0.2625153, 0.34896982, -0.8499285, 1.8921944...",-0.263,0.349,-0.850,1.892,-0.880,0.890,-0.538,-0.588,"[0.42070436, -1.3318422, -0.91565543, 1.518927...","[0.36132258, -0.66763866, -1.1494113, 0.307029..."
4,792-0_10_4,90.000,"(164.0, 678.0)","(164.0, 890.0)",0.000,"(0.0, 0.0)","POLYGON ((199 158, 93 158, 66 132, 0 132, 0 13...",18230.000,0.580,-0.789,0.560,1.192,-1.334,-1.092,0.347,-0.497,-0.493,0.000,1.000,0.239,164.000,678.000,890.000,0.608,-1.112,-0.414,-0.269,"[-0.10381872, -0.3241408, -0.5326116, 1.695041...",-0.104,-0.324,-0.533,1.695,-0.222,0.483,-0.767,0.280,"[0.49004108, -1.522208, -0.18397297, 0.6107793...","[1.3287598, -2.0762594, -1.2859267, -0.8665443..."


## add miss features: width/height

In [111]:
df2 = df2.merge(data[['norm_area', 'width/height']].drop_duplicates(), how = 'left', on = 'norm_area')

In [136]:
df2.to_pickle('../results/all_norm_features_plus_we.pkl')

In [140]:
df2.head()

,norm_area,length,area quote,convex area quote,convex area,area verhaltnis,centroid x/width,centroid y/height,num_of_corr,A/L,type,wv,wv0,wv1,wv2,wv3,wv4,wv5,wv6,wv7,width/height
0,0.916,0.847,-0.514,0.512,1.266,-0.965,0.733,2.238,0.910,0.315,0,"[0.74889624, -2.125927, -1.3886113, -0.3797980...",0.749,-2.126,-1.389,-0.380,-1.620,-0.865,-0.516,-1.622,0.560
1,0.608,0.580,-0.789,0.560,1.192,-1.334,-1.092,0.347,-0.493,0.239,1,"[1.3287598, -2.0762594, -1.2859267, -0.8665443...",1.329,-2.076,-1.286,-0.867,-1.685,-1.174,-0.050,-0.810,-0.497
2,-0.163,0.339,-1.070,-1.456,-0.100,-0.364,-1.348,-0.681,0.008,-0.461,2,"[0.64179367, -2.2796211, -1.4502786, 0.556967,...",0.642,-2.280,-1.450,0.557,-0.525,-0.861,-0.631,-1.927,-0.503
3,0.125,-0.032,-1.078,-1.227,0.252,-0.577,0.396,-1.971,0.409,0.322,3,"[0.36132258, -0.66763866, -1.1494113, 0.307029...",0.361,-0.668,-1.149,0.307,-3.271,1.210,-0.595,-1.533,-0.316
4,1.537,0.050,1.486,0.927,0.709,1.176,0.162,0.102,-1.295,2.214,4,"[0.74881685, -1.5598556, -1.7012521, -0.611160...",0.749,-1.560,-1.701,-0.611,-1.460,-1.383,1.359,-0.945,-0.542


## transform test data
- original test data ist type of list of tuple
- the first number in the tuple should correspont to the type feature in the table above
- the second number in the tuple should be the rotation

In [115]:
# test data for the test data
a = [(33, 0), (41, 90), (2, 0), (22, 0), (16, 90), (31, 0), (23, 0), (11, 0), (2, 0), (34, 90), (26, 0), (19,0), (8, 0), (28, 90), (36, 0), (22, 0), (15, 0)]

In [134]:
def transform_state_to_input(state, df):
    """
    FIXED STRUCTURE!!!
    input: 
      state, type of list of tuple, tuple contains: (type of part, rotation)
        sequence of inputted parts
      df, type of Dataframe, 
        inlucde all the needed features(after norm)
    output:
      out, type of numpy.array 
        data, that can be used as input of the neural network.
    """
    df_tmp = []
    for i in a:
        df_tmp.append(df[df['type'] == i[0]])
    df_tmp = pd.concat(df_tmp)
    df_tmp['rotation'] = [i[1] for i in a]
    df_tmp['r1'] = df_tmp['rotation'].map(lambda x: 0 if x == 90 else 1)
    df_tmp['r2'] = df_tmp['rotation'].map(lambda x: 1 if x == 90 else 0)
    # do features selection to make sure that all the features are included and in a right sequence.
    df_tmp = df_tmp[['norm_area', 'length', 'num_of_corr', 'A/L', 
                            'area quote', 'convex area quote', 'centroid x/width',
                            'centroid y/height', 'width/height', 'convex area','area verhaltnis',
                            'r1', 'r2', 'wv0', 'wv1', 'wv2', 'wv3', 'wv4', 'wv5', 'wv6', 'wv7']]
    out = df_tmp.values.reshape(1, len(a), -1)
    return out

In [132]:
transform_state_to_input(a, df2)

In [139]:
df2['type'].tolist()

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53]

## self defined methods

In [20]:
# define method to get the n combination.
def get_combinations(nc, seq, si = 0, com = []):
    """
    input:
    nc, type of int
        set the number of items in one combination
    seq, type of list
        from whom we get the items
    si, type of int
        only consider the items which index larger than si
    com, type of list
        used for recursion, should be set to [] when calling the function
    """
    out = []
    for i in range(si, len(seq)):
        if seq[i] not in com:
            com.append(seq[i])
        else:
            continue
        if len(com) < nc:
            com, tmp_out = get_combinations(nc, seq, i, com)
            out.extend(tmp_out)
        elif len(com) == nc:
            #print(com)
            out.append(com)
            com = com[:-1]
        else:
            print('ERROR!!!')
            #print(com)
    return com[:-1], out